# **Recommendation Systems**



Geçmiş tercihlere bakılarak kullanıcının bir itemi tercih etme olasılığını tahmin eder.

Mesela Netflix kullanıcıların önceden izledikleri dizi/filmlere göre yeni bir dizi/film önerir.

Kullanıcı tabanlı ve item tabanlı olmak üzere ikiye ayrılır.

**User Based Collaborative Filtering**

Kullanıcı tabanlıdır.

Kullanıcının ve diğer kullanıcıların deneyimleri kombinasyonu alınarak öneri yapılmasıdır.

İlk olarak user ve itemdan oluşan bir matrix yapılır.

Matrix'teki satırlar user, sütunlar itemdır.

İkinci olarak userlar arasındaki benzerlik bir score değeri ile elde edilir.

Üçüncü olarak benzerlik score'una göre yeni kullanıcılar bulunur.

Son olarak yeni kullanıcının daha önce deneyimlemediği bir item ona önerilir.
 
Mesela iki kullanıcı olsun.

Birinci kullanıcı Lord of the Rings ve Hobbit'i izlesin.

İkinci kullanıcı da Lord of the Rings izlesin.

Sistem iki kullanıcı arasındaki benzerliği hesaplar.

İkinci kullanıcıya daha önceden izlemediği Hobbit filmini önerir.

Kullanıcı tabanlı sistemin bazı dezavantajları vardır: 

*   Matrix'te satırlar kullanıcıları temsil etmektedir. Dünyada 7 milyar insan bulunmaktadır. Bu kadar kullanıcı arasındaki benzerliği hesaplamak çok zordur ve çok fazla hesaplama gücü gerekir.
*   Ayrıca insanların alışkanlıkları değişebileceğinden yapılan öneriler yanılabilir. Kullanıcı 5 yıl önce aksiyon filmlerini izlerken şu anda romantik filmler izleyebilir. Ancak sistemde aksiyon tarzı kayıtlı olduğu için sistem bu tarzda öneri yapar.

**Item Based Collaborative Filtering**

Item tabanlıdır.

Itemler arasındaki benzerliğe bakılır.

Kullanıcı tabanlı sistemde insanların alışkanlıkları değişebilir, ancak itemler(filmler/eşyalar) değişmez.

Kullanıcı tabanlı sistemde 7 milyar insan varken, item tabanlıda karşılaştırılacak item sayısı çok daha azdır. Daha az karşılaştırma gücü harcanır.

Bu sistemdeki matrix'te satırlar kullanıcı, sütunlar item'dır.

Sütunların arasındaki benzerlik hesaplanıp kullanıcıya bir öneri sunulur.


In [1]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
movie = pd.read_csv("/content/drive/MyDrive/data/movie.csv")

In [3]:
movie.columns

Index(['movieId', 'title', 'genres'], dtype='object')

Datadaki sütunlar movieId, filmin adı ve türüdür.

Recommendation systemde movieId ve title kullanılacaktır.

In [4]:
movie = movie.loc[:,["movieId","title"]]
movie.head(10)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


In [5]:
rating = pd.read_csv("/content/drive/MyDrive/data/rating.csv")

In [6]:
rating.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

rating datasından userId, movieId ve rating sütunları kullanılacaktır.

In [7]:
rating = rating.loc[:,["userId","movieId","rating"]]
rating.head(10)

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5
5,1,112,3.5
6,1,151,4.0
7,1,223,4.0
8,1,253,4.0
9,1,260,4.0


UserId -> Kullanıcının id'si 1 numaralı kullanıcı

movieId -> filmin id'si

rating -> 5 üzerinden kaç puan verildiği

İki datada da movieId ortaktır. movie ve rating dataları merge methoduyla birleştirilir.

In [8]:
data = pd.merge(movie,rating)

In [9]:
data.head(10)

,movieId,title,userId,rating
0,1,Toy Story (1995),3,4.0
1,1,Toy Story (1995),6,5.0
2,1,Toy Story (1995),8,4.0
3,1,Toy Story (1995),10,4.0
4,1,Toy Story (1995),11,4.5
5,1,Toy Story (1995),12,4.0
6,1,Toy Story (1995),13,4.0
7,1,Toy Story (1995),14,4.5
8,1,Toy Story (1995),16,3.0
9,1,Toy Story (1995),19,5.0


Bu dataframe'e göre item based recommendation system yapılacaktır.

In [10]:
data.shape

(20000263, 4)

Datada 20 milyon sample vardır. Bu da datanın uzunluğunun 20 milyon olduğu anlamına gelir. Bu kadar uzun datayla uğraşmak makineyi çok yorar. Bu yüzden ilk 1 milyon sample alınacaktır.

In [11]:
data = data.iloc[:1000000,:]

Satırlar user, sütunlar film isimleri olacak şekilde bir matrix oluşturulur.

In [12]:
pivot_table = data.pivot_table(index = ["userId"],columns = ["title"],values = "rating")
pivot_table.head(10)

title,Ace Ventura: When Nature Calls (1995),Across the Sea of Time (1995),"Amazing Panda Adventure, The (1995)","American President, The (1995)",Angela (1995),Angels and Insects (1995),Anne Frank Remembered (1995),Antonia's Line (Antonia) (1995),Assassins (1995),Babe (1995),...,Unforgettable (1996),Up Close and Personal (1996),"Usual Suspects, The (1995)",Vampire in Brooklyn (1995),Waiting to Exhale (1995),When Night Is Falling (1995),"White Balloon, The (Badkonake sefid) (1995)",White Squall (1996),Wings of Courage (1995),"Young Poisoner's Handbook, The (1995)"
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Bad Boys filmini izleyen kullanıcılara hangi filmin önerileceğine dair bir recommendation system yapılacaktır.

Bunun için Bad Boys filmi ile geri kalan tüm filmler arasındaki benzerliğe bakılacaktır.

Benzerliği yüksek olanlar Bad Boys filminden sonra tavsiye edilecek filmler olacaktır.

In [13]:
movie_watched = pivot_table["Bad Boys (1995)"] #Bad Boys sütunu movie_watched adlı variable'a eşitlenir.
similarity_with_other_movies = pivot_table.corrwith(movie_watched)  
#pivot_table'daki filmler ile Bad Boys filmi arasındaki korelasyona corrwith methoduyla bakılır.
similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False) 
#sort_values methoduyla ascending=False parametresiyle azalan sırada yukarıdan aşağı listeler.
similarity_with_other_movies.head(10)

title
Bad Boys (1995)                        1.000000
Headless Body in Topless Bar (1995)    0.723747
Last Summer in the Hamptons (1995)     0.607554
Two Bits (1995)                        0.507008
Shadows (Cienie) (1988)                0.494186
Guardian Angel (1994)                  0.465218
Money Train (1995)                     0.451802
Across the Sea of Time (1995)          0.450071
Assassins (1995)                       0.427445
Vampire in Brooklyn (1995)             0.426861
dtype: float64

Bu karşılaştırmaya bakılarak Bad Boys filmini izleyen birine ilk olarak Headless Body in Topless Bar filmi önerilir.